In [1]:
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
tf.test.is_built_with_cuda()

True

### https://keras.io/api/datasets/cifar10/

In [5]:
(X_train, y_train), (X_test, y_test) = keras.datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [6]:
X_train.shape, y_train.shape

((50000, 32, 32, 3), (50000, 1))

In [7]:
X_test.shape, y_test.shape

((10000, 32, 32, 3), (10000, 1))

In [8]:
np.unique(y_test, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
 array([1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]))

# Data Manipulation

In [9]:
X_train_scaled = X_train / 255
X_test_scaled = X_test /255

In [10]:
y_train_encoded = keras.utils.to_categorical(y_train, num_classes=10)
y_test_encoded = keras.utils.to_categorical(y_test, num_classes=10)

y_train_encoded.shape, y_train_encoded

((50000, 10),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 0., 0., ..., 0., 0., 1.],
        ...,
        [0., 0., 0., ..., 0., 0., 1.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.]]))

# Build Model

In [11]:
def nn_adam():
    model = keras.Sequential([
        keras.layers.Input(shape=(32, 32, 3)),  # 32x32x3 = 3072
        keras.layers.Flatten(),
        keras.layers.Dense(300, activation='relu'),
        keras.layers.Dense(10, activation='softmax')])

    model.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    return model

In [12]:
def nn_sgd():
    model = keras.Sequential([
        keras.layers.Input(shape=(32, 32, 3)),  # 32x32x3 = 3072
        keras.layers.Flatten(),
        keras.layers.Dense(300, activation='relu'),
        keras.layers.Dense(10, activation='softmax')])

    model.compile(optimizer='SGD',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])

    return model

# Train Model on CPU

In [13]:
%%timeit -n1 -r1  # to display the time it took to run
with tf.device('/CPU:0'):
    model_on_CPU = nn_sgd()
    model_on_CPU.fit(X_train_scaled, y_train_encoded, epochs=5)

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 15s 8ms/step - accuracy: 0.2822 - loss: 1.9800
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - accuracy: 0.3954 - loss: 1.7142
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.4288 - loss: 1.6306
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.4482 - loss: 1.5802
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.4630 - loss: 1.5324
55.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [14]:
%%timeit -n1 -r1  # to display the time it took to run
with tf.device('/CPU:0'):
    model_on_CPU = nn_adam()
    model_on_CPU.fit(X_train_scaled, y_train_encoded, epochs=5)

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.2856 - loss: 2.0147
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.3740 - loss: 1.7424
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.3996 - loss: 1.6766
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 10s 5ms/step - accuracy: 0.4164 - loss: 1.6364
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.4328 - loss: 1.5993
51.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


---

# Train Model on GPU

In [15]:
%%timeit -n1 -r1  # to display the time it took to run
with tf.device('/GPU:0'):
    model_on_CPU = nn_sgd()
    model_on_CPU.fit(X_train_scaled, y_train_encoded, epochs=5)

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.2871 - loss: 1.9823
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.3870 - loss: 1.7348
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4247 - loss: 1.6407
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.4429 - loss: 1.5846
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4605 - loss: 1.5437
17.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [16]:
%%timeit -n1 -r1  # to display the time it took to run
with tf.device('/GPU:0'):
    model_on_CPU = nn_adam()
    model_on_CPU.fit(X_train_scaled, y_train_encoded, epochs=5)

Epoch 1/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.2853 - loss: 2.0540
Epoch 2/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.3899 - loss: 1.7134
Epoch 3/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.4123 - loss: 1.6534
Epoch 4/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.4266 - loss: 1.6182
Epoch 5/5
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.4397 - loss: 1.5820
24 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
